<a href="https://colab.research.google.com/github/masunomatiko/lab/blob/master/%E3%83%A9%E3%83%B3%E3%82%AF%E5%AD%A6%E7%BF%92%E3%81%AE%E6%B0%97%E6%8C%81%E3%81%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ランク学習の気持ち

In [0]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

## 0. クエリと文書のマッチングスコアリング手法

### Tf-Idf
**各文書を特徴付ける単語ほど大きくなる**<br>


ある単語iについて、<br>
Tf-Idf =（文書内での出現頻度）*（文書集合での出現頻度）

In [0]:
# sklearnから使える
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

vectorizer = TfidfVectorizer(max_df=0.5, min_df=4)
data = vectorizer.fit_transform(newsgroups_train.data[:1000]).toarray()
name = np.array(vectorizer.get_feature_names())

In [0]:
index = data.argsort(axis=1)[:,::-1]
word =name[index]

In [0]:
for w, target in zip(word[:10, :5], newsgroups_train.target):
    # 各文書ごとにtarget（ラベル）とtop nの重要語を表示
    print('正解ラベル', newsgroups_train.target_names[target].replace('.', ' '))
    print(w)
    print('')

正解ラベル rec autos
['car' 'umd' 'was' '70s' 'funky']

正解ラベル comp sys mac hardware
['clock' 'si' 'washington' 'carson' 'kuo']

正解ラベル comp sys mac hardware
['180' 'purdue' 'powerbook' 'display' 'willis']

正解ラベル comp graphics
['harris' 'jgreen' 'iastate' 'csd' 'green']

正解ラベル sci space
['errors' 'warning' 'bugs' 'std' 'launch']

正解ラベル talk politics guns
['weapons' 'destruction' 'mass' 'stratus' 'cdt']

正解ラベル sci med
['uchicago' 'treatment' 'brian' 'thank' 'rn']

正解ラベル comp sys ibm pc hardware
['scsi' 'chip' 'mac' 'ranges' 'nmsu']

正解ラベル comp os ms-windows misc
['win' 'help' 'bmp' 'icon' 'iowa']

正解ラベル comp sys mac hardware
['board' 'licensing' 'file' 'stan' 'designs']



### BM25
TfIdfに加えて、**より短い文書において出現頻度の高い単語**ほど大きくなる<br>
NDL = （文書の単語数） \* （文書集合の平均単語数）<br>
ある単語について、<br>
CW = (TF \* IDF \* (k1 + 1) ) / (k1 \* (1 - b + (b \* NDL) + TF))



In [0]:
# gensimから使える
from gensim.summarization.bm25 import get_bm25_weights
corpus = []
for text in newsgroups_train.data[:1000]:
  corpus.append(text.split())

In [0]:
results = get_bm25_weights(corpus, n_jobs=-1)
results[0][:20]

[500.12099319773563,
 74.19921994367022,
 94.20017310724802,
 61.02839301236178,
 76.00875423106261,
 81.68202141201162,
 59.396554023466756,
 78.6700722254541,
 55.47459872606066,
 81.14580804047165,
 81.71106858765052,
 105.924200073023,
 15.524192936203445,
 78.54255785694188,
 69.88936990985424,
 79.0514122142424,
 96.46020025185516,
 114.68133734762905,
 90.18225139010069,
 73.96990098226352]

## 1. 学習データ

「クエリ」「クエリに対応する検索結果リスト」が学習データとして与えられる。<br>
検索結果リストの各文書dには、クエリqと文書dの関連度を表すラベルyが付与されている。また、クエリqと文書dのペアから特徴量xが作られる。<br>


実際に使う学習データはデータフレームからこんな感じで作る

In [1]:
import pandas as pd

df = pd.DataFrame([[4, 0.2, 0.4, 0.2], [1, 0.1, 0.14, 0.02]], index=['new', 'old'], columns=['score', 'f1', 'f2', 'f3'])
df

,score,f1,f2,f3
new,4,0.2,0.40,0.20
old,1,0.1,0.14,0.02


In [0]:
df['f1'] = list(map(lambda x: '1:'+str(x), df['f1']))
df['f2'] = list(map(lambda x: '2:'+str(x), df['f2']))
df['f3'] = list(map(lambda x: '3:'+str(x), df['f3']))
df.to_csv('lambdarank.csv', index=False, header=False, sep=' ')

こんな感じのファイルができる

左端がラベルy、続く数字が特徴量のラベル、コロンのあとは特徴量の値
```
4 1:0.2 2:0.4 3:0.2
1 1:0.1 2:0.14 3:0.02
```

## 2. 特徴量
### 文書から得られる特徴量
文書の単語数とか、特定の単語が文書に含まれるか(BoW)とか
### クエリから得られる特徴量
クエリの長さとか、ログにおけるクエリの出現頻度とか
### 文書とクエリのペアから得られる特徴量
TfIdfとか、BM25とか
### そのほかにも、、
パーソナライズするならユーザのデモグラフィック情報とか、位置情報とか

## 3. 損失関数
### ポイントワイズ
学習データ(x, y)からランキングモデルf(x)から出力するy^がラベルyを正確に当てることができれば検索結果のランキングも正しいものになるという仮定。


ポイントワイズの損失関数は、<br>
**L0 = ΣL( f(x), y ) **<br>で定義される。

ここで、f(x)は分類問題、回帰問題のようにクエリと文書の関連度を予測するモデルではないので、ランキングが合っていれば損失は0である。

![](https://cdn-ak.f.st-hatena.com/images/fotolife/s/sz_dr/20181204/20181204002443.png)
[ランク学習ってどうやって学習するの？学習データ・特徴量・損失関数](http://szdr.hatenablog.com/entry/2018/12/04/010611)から引用させていただきました

>*もし単純に二乗誤差を考えると大きい損失が出てきますが、今回出力したスコアでも検索結果ランキングとしては正しい（スコアの順に文書をランキングすれば、関連度の順に並ぶ）ので、ランキングという観点から考えると損失は0であるべきです。*

### ペアワイズ
2つの文書ペアを正しく順序づけできれば、検索結果は正しいランキングになると仮定。

ペアワイズの損失関数は、<br>**L0 = Σ_q Σ_i,j L( f(xi), f(xj) )** (ただし、yi > yj )<br>定義される。

　### リストワイズ
 ランキング問題の評価指標において、検索結果のリスト全体としていい並び順になっているかどうかを指標として計算するという考え方。
 
 リストワイズの損失関数は、<br>**L0 = Σ L( ( f(x1), y1 ), ... , ( f(xm), ym ) ) )**<br>で定義される。
 
 例えばランキング問題の評価指標としてよく用いられるnDCGを直的最適化するとか（実際は緩和した問題を解くアルゴリズムが提案されている）。